## Requirements
- The XRADIO images are stored xr.Dataset that remains compatible with Xarray
    - Xarray API should work with structure (to_zarr, open_dataset, open_zarr)
        - No numpy data structures in attributes (otherwise serialization fails). Use list and standard numerical types.
        - No other xarray data structures in attributes (look into storing history as just a dict).
    - Allow multiple images and image types as long as they share coordinates.
- Make use of the same convention as Measurement Set Module/Schema
    - `read_image` renamed to `open_image`
    - All measures should be formatted as Xarray-compatible dicts. For example:
    ```Python
        reference_direction = {'attrs': {'frame': 'fk5',
                                'type': 'sky_coord',
                                'units': 'rad',
                                'equinox': 'j2000.0'},
            'data': [3.5392577860590637, 0.5324852164475642],
            'dims': "sky_dir_label",
            'coords': {"sky_dir_label":
                            {"data":["ra","dec"]
                            "dims":"sky_dir_label"}
                    }
        }
    ```
    - Try reusing the coordinate schemas from the measurement set schema (for example, frequency coordinate).
    - Add frame to location called native_projection for native_pole_direction (latpole and lonpole in fits). latpole and lonpole is combined in a single measure called native_pole_direction.
    - Renamed BEAM to BEAM_FIT_PARAMS and beam_param to beam_param_label.
    - Add types to all data variables and dataset.

## Image Types
    sky
    flag
    point_spread_function
    primary_beam
    mask
    beam_fit_params_sky
    beam_fit_params_point_spread_function
    visibility_normalization
    description:
    date:

    visibility*
    visibility_normalization*
    uv_sampling*
    uv_sampling_normalization*
    aperture*
    aperture_normalization*

* UV plain. Only used internally and for debugging.
    
## Questions:
- What is the use case for multiple rest_frequencies (this only seems to be a thing for FITS images)?
- If the addition of datagroups, can we depricate active_mask?
- How to handle the different types of masks:
    - External Mask added as just another image.
    - Internal Masks are currently added by appending the image they come from. For example, MASK_RESIDUAL, MASK_SKY.

## Issues
- Visibility normalization is not loaded correctly (sum_weight). Adds unnecessary l and m coordinates and tiles. 

## Measures example

In [1]:
import xarray as xr 

reference_direction = {'attrs': {'frame': 'fk5',
                        'type': 'sky_coord',
                        'units': 'rad',
                        'equinox': 'j2000.0'},
    'data': [3.5392577860590637, 0.5324852164475642],
    'dims': "sky_dir_label",
    'coords': {"sky_dir_label":
                    {"data":["ra","dec"],
                     "dims":"sky_dir_label"}
            }
}

dir_xda = xr.DataArray.from_dict(reference_direction)
dir_xda

<xarray.DataArray (sky_dir_label: 2)> Size: 16B
array([3.53925779, 0.53248522])
Coordinates:
  * sky_dir_label  (sky_dir_label) <U3 24B 'ra' 'dec'
Attributes:
    frame:    fk5
    type:     sky_coord
    units:    rad
    equinox:  j2000.0

## Download Data

In [2]:
import toolviper

toolviper.utils.data.update()
# toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits")
# toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa")

toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99.fits")
toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa.ms") #Not an ms. Mistake in download metadata. CASA images.

casa_dir = "3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/"
fits_dir = "3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits/"

[2026-01-14 15:58:04,597]     INFO    viperlog:  Updating file metadata information ...  


Output()

[2026-01-14 15:58:04,836]     INFO    viperlog:  Downloading from [cloudflare] .... 


 Download List                                     
 ────────────────────────────────────────────────── 
  3c286_Band6_5chans_lsrk_robust_0.5_niter_99.fits

Output()

[2026-01-14 15:58:05,530]     INFO    viperlog:  Downloading from [cloudflare] .... 


 Download List                                        
 ───────────────────────────────────────────────────── 
  3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa.ms

Output()

# Load and Open a Single Image

In [3]:
#Opening a single casa or fits image into an xarray Dataset

from toolviper.utils.display import dict_to_html
from IPython.display import HTML, display

from xradio.image import open_image, load_image
import toolviper.utils.logger as logger

casa_dir = "3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/"
fits_dir = "3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits/"

img_xds = open_image(
        fits_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image.fits",
) #Opening a FITS image 

img_xds = open_image(
        casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
)#Opening a CASA image 

img_xds

Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image: 1 columns, 1 rows


<xarray.Dataset> Size: 7MB
Dimensions:              (time: 1, frequency: 5, polarization: 4, l: 250,
                          m: 250, beam_params_label: 3)
Coordinates:
  * time                 (time) float64 8B 5.684e+04
  * frequency            (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization         (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                    (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                    (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label    (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity             (frequency) float64 40B 2.216e+05 ... -4.03e+05
    right_ascension      (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination          (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY  (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    FLAG_SKY             (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image_dataset
    data_groups:             {'base': {'sky': 'SKY', 'beam_fit_params_sky': '...

In [4]:
display(HTML(dict_to_html(img_xds.attrs)))

In [5]:
display(HTML(dict_to_html(img_xds.SKY.attrs)))

In [6]:
#Loading a single casa or fits image into an xarray Dataset

from toolviper.utils.display import dict_to_html
from IPython.display import HTML, display

from xradio.image import open_image, load_image
import toolviper.utils.logger as logger


img_xds = load_image(
        casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
)

img_xds

Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image: 1 columns, 1 rows


<xarray.Dataset> Size: 7MB
Dimensions:              (time: 1, frequency: 5, polarization: 4, l: 250,
                          m: 250, beam_params_label: 3)
Coordinates:
  * time                 (time) float64 8B 5.684e+04
  * frequency            (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization         (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                    (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                    (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label    (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity             (frequency) float64 40B 2.216e+05 ... -4.03e+05
    right_ascension      (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination          (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY  (time, frequency, polarization, beam_params_label) float64 480B ...
    FLAG_SKY             (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image_dataset
    data_groups:             {'base': {'sky': 'SKY', 'beam_fit_params_sky': '...

## Loading Mutiple CASA and FITS Images

- VISIBILITY_NORMALIZATION is incorrectly loaded.
- Can mix FITS and CASA images (still have to share coordinates and have the same Dataset level attributes). 

In [7]:
from xradio.image import open_image

casa_dir = "3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/"
fits_dir = "3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits/"

img_xds = open_image(    
    {"sky_deconvolved": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
    "sky_dirty": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
    "sky_model": fits_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.model.fits",
    "sky_residual": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.residual",
    "mask": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.mask",
    "primary_beam": fits_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.pb.fits",
    "point_spread_function": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.psf",
    "visibility_normalization": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.sumwt"})

img_xds

Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image: 1 columns, 1 rows
Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image: 1 columns, 1 rows
Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.residual: 1 columns, 1 rows
Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.mask: 1 columns, 1 rows
Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.psf: 1 columns, 1 rows
Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.sumwt: 1 columns, 1 row

/Users/jsteeb/miniforge3/envs/timg/lib/python3.13/site-packages/dask/array/core.py:4997: PerformanceWarning: Increasing number of chunks by factor of 250
  result = blockwise(


<xarray.Dataset> Size: 45MB
Dimensions:                                (time: 1, frequency: 5,
                                            polarization: 4, l: 250, m: 250,
                                            beam_params_label: 3)
Coordinates:
  * time                                   (time) float64 8B 5.684e+04
  * frequency                              (frequency) float64 40B 2.398e+11 ...
  * polarization                           (polarization) <U1 16B 'I' ... 'V'
  * l                                      (l) float64 2kB 6.06e-05 ... -6.01...
  * m                                      (m) float64 2kB -6.06e-05 ... 6.01...
  * beam_params_label                      (beam_params_label) <U5 60B 'major...
    velocity                               (frequency) float64 40B 2.216e+05 ...
    right_ascension                        (l, m) float64 500kB 3.539 ... 3.539
    declination                            (l, m) float64 500kB 0.5324 ... 0....
Data variables: (12/14)
    SKY_DECONVOLVED                        (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY_DECONVOLVED        (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    FLAG_SKY_DECONVOLVED                   (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_DIRTY                              (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY_DIRTY              (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    FLAG_SKY_DIRTY                         (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    ...                                     ...
    FLAG_SKY_RESIDUAL                      (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    MASK                                   (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    PRIMARY_BEAM                           (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    POINT_SPREAD_FUNCTION                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS_POINT_SPREAD_FUNCTION  (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    VISIBILITY_NORMALIZATION               (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 1, 1), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image_dataset
    data_groups:             {'deconvolved': {'sky': 'SKY_DECONVOLVED', 'beam...

In [8]:
from toolviper.utils.display import dict_to_html
from IPython.display import HTML, display

display(HTML(dict_to_html(img_xds.attrs)))

## Writing to Zarr

In [9]:
import xarray as xr
if "VISIBILITY_NORMALIZATION" in img_xds.data_vars:
    del img_xds["VISIBILITY_NORMALIZATION"] # remove incorrectly loaded dataarray
img_xds.to_zarr("test_zarr_write.zarr", mode="w")
img_write_test_open = xr.open_zarr("test_zarr_write.zarr")
img_write_test_open

<xarray.Dataset> Size: 40MB
Dimensions:                                (time: 1, frequency: 5,
                                            polarization: 4,
                                            beam_params_label: 3, l: 250, m: 250)
Coordinates:
  * time                                   (time) float64 8B 5.684e+04
  * frequency                              (frequency) float64 40B 2.398e+11 ...
  * polarization                           (polarization) <U1 16B 'I' ... 'V'
  * beam_params_label                      (beam_params_label) <U5 60B 'major...
  * l                                      (l) float64 2kB 6.06e-05 ... -6.01...
  * m                                      (m) float64 2kB -6.06e-05 ... 6.01...
    declination                            (l, m) float64 500kB dask.array<chunksize=(125, 250), meta=np.ndarray>
    right_ascension                        (l, m) float64 500kB dask.array<chunksize=(125, 250), meta=np.ndarray>
    velocity                               (frequency) float64 40B dask.array<chunksize=(5,), meta=np.ndarray>
Data variables: (12/13)
    BEAM_FIT_PARAMS_POINT_SPREAD_FUNCTION  (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY_DECONVOLVED        (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY_DIRTY              (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    FLAG_SKY_DECONVOLVED                   (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    FLAG_SKY_DIRTY                         (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    FLAG_SKY_RESIDUAL                      (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    ...                                     ...
    POINT_SPREAD_FUNCTION                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    PRIMARY_BEAM                           (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_DECONVOLVED                        (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_DIRTY                              (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_MODEL                              (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_RESIDUAL                           (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'native_pole_direction': {'attrs': {'frame': 'N...
    data_groups:             {'deconvolved': {'beam_fit_params_point_spread_f...
    type:                    image_dataset

## Writing to CASA

- Need to implement a better convention for file extensions.

In [10]:
from xradio.image import open_image, write_image
write_image(img_xds, "test_casa_write", overwrite=True)
img_xds_written = open_image("test_casa_write.sky")
img_xds_written

Successful readonly open of default-locked table test_casa_write.sky: 1 columns, 1 rows


<xarray.Dataset> Size: 7MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 2.216e+05 6.54e+04 ... -4.03e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    FLAG_SKY           (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image_dataset
    data_groups:             {'base': {'sky': 'SKY', 'flag': 'FLAG_SKY'}}

## Create an empty image

In [11]:
from xradio.image import make_empty_sky_image

phase_center = img_xds.attrs['coordinate_system_info']['reference_direction']['data']
image_shape = img_xds.SKY_DECONVOLVED.shape
cell_size = [(img_xds.l[1]-img_xds.l[0]).values.item(), (img_xds.m[1]-img_xds.m[0]).values.item()]
frequency_coords = img_xds.frequency.values.tolist()
pol_coords = img_xds.polarization.values.tolist()
time_coords = img_xds.time.values.tolist()

empty_img_xds = make_empty_sky_image(phase_center=phase_center,
    image_size=image_shape[-2:],
    cell_size=cell_size,
    frequency_coords=frequency_coords,
    pol_coords=pol_coords,
    time_coords=time_coords,
    direction_reference = "fK5",
    projection = "SIN",
    spectral_reference = "lsrk",
    do_sky_coords = True,
)

empty_img_xds

<xarray.Dataset> Size: 1MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 3.122e+05 ... -3.122e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    *empty*
Attributes:
    data_groups:             {'base': {}}
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image

## Image Accessor

In [12]:
#Loading a single casa or fits image into an xarray Dataset
from xradio.image import open_image


img_xds = open_image(
        casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
)

img_xds

Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image: 1 columns, 1 rows


<xarray.Dataset> Size: 7MB
Dimensions:              (time: 1, frequency: 5, polarization: 4, l: 250,
                          m: 250, beam_params_label: 3)
Coordinates:
  * time                 (time) float64 8B 5.684e+04
  * frequency            (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization         (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                    (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                    (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label    (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity             (frequency) float64 40B 2.216e+05 ... -4.03e+05
    right_ascension      (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination          (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY  (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    FLAG_SKY             (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image_dataset
    data_groups:             {'base': {'sky': 'SKY', 'beam_fit_params_sky': '...

In [13]:
#Add a new data group with improved image

img_xds["SKY_IMPROVED"] = img_xds["SKY"] * 1.1  # Example improvement: scale the image data


img_xds.xr_img.add_data_group(
        new_data_group_name = "improved",
        new_data_group = {
            "sky": "SKY_IMPROVED",
            "date": "2024-06-10",
            "description": "This data group contains improved image.",
        },
        data_group_dv_shared_with="base"
)

img_xds

<xarray.Dataset> Size: 12MB
Dimensions:              (time: 1, frequency: 5, polarization: 4, l: 250,
                          m: 250, beam_params_label: 3)
Coordinates:
  * time                 (time) float64 8B 5.684e+04
  * frequency            (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization         (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                    (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                    (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label    (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity             (frequency) float64 40B 2.216e+05 ... -4.03e+05
    right_ascension      (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination          (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY  (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    FLAG_SKY             (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_IMPROVED         (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image_dataset
    data_groups:             {'base': {'sky': 'SKY', 'beam_fit_params_sky': '...

In [14]:
display(HTML(dict_to_html(img_xds.attrs["data_groups"])))

In [15]:
#Select the improved data group

img_improved_xds = img_xds.xr_img.sel(data_group_name="improved")
img_improved_xds

<xarray.Dataset> Size: 7MB
Dimensions:              (time: 1, frequency: 5, polarization: 4, l: 250,
                          m: 250, beam_params_label: 3)
Coordinates:
  * time                 (time) float64 8B 5.684e+04
  * frequency            (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization         (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                    (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                    (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label    (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity             (frequency) float64 40B 2.216e+05 ... -4.03e+05
    right_ascension      (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination          (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    BEAM_FIT_PARAMS_SKY  (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    FLAG_SKY             (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_IMPROVED         (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image_dataset
    data_groups:             {'improved': {'sky': 'SKY_IMPROVED', 'flag': 'FL...

In [16]:
img_xds

<xarray.Dataset> Size: 12MB
Dimensions:              (time: 1, frequency: 5, polarization: 4, l: 250,
                          m: 250, beam_params_label: 3)
Coordinates:
  * time                 (time) float64 8B 5.684e+04
  * frequency            (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization         (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                    (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                    (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label    (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity             (frequency) float64 40B 2.216e+05 ... -4.03e+05
    right_ascension      (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination          (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY  (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    FLAG_SKY             (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_IMPROVED         (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image_dataset
    data_groups:             {'base': {'sky': 'SKY', 'beam_fit_params_sky': '...

In [17]:
img_xds.xr_img.get_reference_pixel_indices()

array([125, 125])

In [18]:
img_xds.xr_img.get_lm_cell_size()

array([-4.84813681e-07,  4.84813681e-07])

In [19]:
img_xds = img_xds.xr_img.add_uv_coordinates()
img_xds

<xarray.Dataset> Size: 12MB
Dimensions:              (time: 1, frequency: 5, polarization: 4, l: 250,
                          m: 250, beam_params_label: 3, u: 250, v: 250)
Coordinates:
  * time                 (time) float64 8B 5.684e+04
  * frequency            (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization         (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                    (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                    (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label    (beam_params_label) <U5 60B 'major' 'minor' 'pa'
  * u                    (u) float64 2kB 1.031e+06 1.023e+06 ... -1.023e+06
  * v                    (v) float64 2kB -1.031e+06 -1.023e+06 ... 1.023e+06
    velocity             (frequency) float64 40B 2.216e+05 ... -4.03e+05
    right_ascension      (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination          (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS_SKY  (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    FLAG_SKY             (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_IMPROVED         (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image_dataset
    data_groups:             {'base': {'sky': 'SKY', 'beam_fit_params_sky': '...

In [20]:
img_xds.xr_img.get_uv_in_lambda(frequency=2.399476e+11)

(<xarray.DataArray 'u' (u: 250)> Size: 2kB
 array([ 8.25450139e+08,  8.18846538e+08,  8.12242937e+08,  8.05639336e+08,
         7.99035735e+08,  7.92432133e+08,  7.85828532e+08,  7.79224931e+08,
         7.72621330e+08,  7.66017729e+08,  7.59414128e+08,  7.52810527e+08,
         7.46206926e+08,  7.39603325e+08,  7.32999723e+08,  7.26396122e+08,
         7.19792521e+08,  7.13188920e+08,  7.06585319e+08,  6.99981718e+08,
         6.93378117e+08,  6.86774516e+08,  6.80170915e+08,  6.73567313e+08,
         6.66963712e+08,  6.60360111e+08,  6.53756510e+08,  6.47152909e+08,
         6.40549308e+08,  6.33945707e+08,  6.27342106e+08,  6.20738505e+08,
         6.14134903e+08,  6.07531302e+08,  6.00927701e+08,  5.94324100e+08,
         5.87720499e+08,  5.81116898e+08,  5.74513297e+08,  5.67909696e+08,
         5.61306094e+08,  5.54702493e+08,  5.48098892e+08,  5.41495291e+08,
         5.34891690e+08,  5.28288089e+08,  5.21684488e+08,  5.15080887e+08,
         5.08477286e+08,  5.01873684e+08,  4.

In [21]:
img_xds.xr_img.get_reference_pixel_indices()

array([125, 125])

## ?CASA Image Analysis Functions Mapping to RADPS?

- XRADIO Accessors
    - imhead (summary) — summarize and manipulate the “header” information in a CASA image

- XRADIO provided
    - importfits — import a FITS image into a CASA image format table
    - exportfits — write out an image in FITS format

- Use native Xarray
    - imsubimage — Create a (sub)image from a region of the image
    - immath — perform mathematical operations on or between images
    - imval — extract the data and mask values from a pixel or region of an image
    - imtrans — reorder the axes of an image or cube
    - imcollapse — collapse image along one or more axes by aggregating pixel values along that axis
    - makemask — image mask handling

- AstroVIPER
    - xds in, results dict
        - imstat — calculate statistics on an image or part of an image
        - imfit — image plane Gaussian component fitting #component dataset?
        - specfit — fit 1-dimensional Gaussians, polynomial, and/or Lorentzians models to an image or image region
        - specflux — Report details of an image spectrum.
        - spxfit — Calculation of Spectral Indices and higher order polynomials

    - xds in, xds out (same coordinates)
        - imcontsub — perform continuum subtraction on a spectral-line image cube
        - immoments — compute the moments of an image cube
        - specsmooth — 1-dimensional smooth images in the spectral and angular directions
        - imsmooth — 2-dimensional smooth images in the spectral and angular directions
        - rmfit — Calculation of rotation measures

    - xds in, xds out (coordinates changed, new dataset)
        - imregrid — regrid an image onto the coordinate system of another image
        - imreframe — change the frame in which the image reports its spectral values
        - imrebin — rebin an image
        - impv — generate a position-velocity diagram along a slit

    - plotting
        - plotprofilemap — Plot spectra at their position


- Miscellaneous
    - slsearch — query a subset of the Splatalogue spectral line catalog
    - splattotable — convert a file exported from Splatalogue to a CASA table


```
def image_statistics(image_name: str, mask_name: str): #imstat
    stats_dict = {}
    return stats_dict


#User can specify input and output image selections in two ways:
input_image_sel= {"sky":"SKY","mask":"MASK"}
output_image_sel= {"sky":"SKY_SMOOTHED","mask":"MASK"}
or
input_image_sel= {"data_group_name":"base"}
output_image_sel= {"data_group_name":"smoothed"}

def image_smooth(img_xds, input_image_sel, output_image_sel): #imsmooth
    if "data_group_name" in input_image_sel:
        data_group = img_xds.attr["data_groups"][input_image_sel["data_group_name"]]
    else:
        data_group = input_image_sel


    return img_xds
```